In [1]:
%cd ..

/Users/juansegundohevia/Documents/Rice MDS/ELEC631/quantized-education


In [2]:
# load a model
from mlx_lm import load, generate
import torch
import utils as lora_utils

model, tokenizer = load("mlx-community/Llama-3.2-1B-Instruct-bf16")

/Users/juansegundohevia/anaconda3/envs/mlx/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 79891.50it/s]


In [101]:
import importlib
importlib.reload(lora_utils)

<module 'utils' from '/Users/juansegundohevia/Documents/Rice MDS/ELEC631/quantized-education/lora/utils.py'>

In [5]:
orig_mods = model.named_modules()

In [6]:
len(orig_mods)

212

In [80]:
orig_mods

[('',
  Model(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 2048)
      (layers.0): TransformerBlock(
        (self_attn): Attention(
          (q_proj): Linear(input_dims=2048, output_dims=2048, bias=False)
          (k_proj): Linear(input_dims=2048, output_dims=512, bias=False)
          (v_proj): Linear(input_dims=2048, output_dims=512, bias=False)
          (o_proj): Linear(input_dims=2048, output_dims=2048, bias=False)
          (rope): DynamicNTKScalingRoPE()
        )
        (mlp): MLP(
          (gate_proj): Linear(input_dims=2048, output_dims=8192, bias=False)
          (down_proj): Linear(input_dims=8192, output_dims=2048, bias=False)
          (up_proj): Linear(input_dims=2048, output_dims=8192, bias=False)
        )
        (input_layernorm): RMSNorm(2048, eps=1e-05)
        (post_attention_layernorm): RMSNorm(2048, eps=1e-05)
      )
      (layers.1): TransformerBlock(
        (self_attn): Attention(
          (q_proj): Linear(input_dims=2048, output_d

In [89]:
orig_mods[3]

('model.layers.15',
 TransformerBlock(
   (self_attn): Attention(
     (q_proj): Linear(input_dims=2048, output_dims=2048, bias=False)
     (k_proj): Linear(input_dims=2048, output_dims=512, bias=False)
     (v_proj): Linear(input_dims=2048, output_dims=512, bias=False)
     (o_proj): Linear(input_dims=2048, output_dims=2048, bias=False)
     (rope): DynamicNTKScalingRoPE()
   )
   (mlp): MLP(
     (gate_proj): Linear(input_dims=2048, output_dims=8192, bias=False)
     (down_proj): Linear(input_dims=8192, output_dims=2048, bias=False)
     (up_proj): Linear(input_dims=2048, output_dims=8192, bias=False)
   )
   (input_layernorm): RMSNorm(2048, eps=1e-05)
   (post_attention_layernorm): RMSNorm(2048, eps=1e-05)
 ))

In [47]:
RANK_SIZE = 16

In [173]:
from copy import deepcopy

newmodel = deepcopy(model)

In [174]:
importlib.reload(lora_utils)
newmodel = lora_utils.replace_mlp_with_lora(newmodel, rank_size=RANK_SIZE, layers_to_replace=[0]) # replace at model.layers.15.mlp

In [167]:
newmodel.layers[0].mlp.gate_proj = lora_utils.LoRALayer(newmodel.layers[0].mlp.gate_proj, rank_size=RANK_SIZE)

In [175]:
print(newmodel.layers[0].mlp)

LoRAMLP(
  (layers): ModuleDict(
    (gate_proj): LoRALayer()
    (down_proj): LoRALayer()
    (up_proj): LoRALayer()
  )
)


In [169]:
print(newmodel.layers[0].mlp.gate_proj)

LoRALayer()
